$$\frac{1}{r}\frac{\partial}{\partial r} r\frac{\partial u}{\partial r} = 0$$

$$\Rightarrow r\frac{\partial u}{\partial r} = c_1,$$

$$\Rightarrow u = c_1\ln r + c_2;$$

$$u(R) = 0 \Rightarrow c_2 = -c_1\ln R,$$

$$\Rightarrow u = c\ln \frac{R}{r}.$$

The free boundary constraints are

$$u(s) = g(s), \quad \frac{\partial u}{\partial r}(s) = \frac{\partial g}{\partial r}(s),$$

a system of 2 eqns in 2 unknowns for $c$ and $s$.
We'll assume that

$$g = a\max\{0, 1 - (r / \rho)^2\},$$

so

$$g' = \begin{cases} -2ar / \rho^2 & r < \rho \\ 0 & \end{cases}$$

and right away we can guess that $s < \rho.$

$$u(s) = c\ln\frac{R}{s} = a(1 - (s / \rho)^2) = g(s)$$

$$u'(s) = -\frac{c}{s} = -2as/\rho^2 = g'(s)$$

$$\Rightarrow c = 2as^2/\rho^2$$

$$\Rightarrow \frac{2s^2}{\rho^2}\ln\frac{R}{s} + \frac{s^2}{\rho^2} = 1$$

$$\Rightarrow \ln\frac{R^2}{s^2} = \frac{\rho^2}{s^2} - 1$$

In [ ]:
import firedrake
from firedrake import inner, sqrt, Constant, interpolate, project

def create_geometry(num_levels, radius):
    mesh = firedrake.UnitDiskMesh(num_levels)
    V = firedrake.VectorFunctionSpace(mesh, "CG", 2)
    R = Constant(radius)
    x = interpolate(R * firedrake.SpatialCoordinate(mesh), V)
    z = R * x / sqrt(inner(x, x))
    bc = firedrake.DirichletBC(V, z, "on_boundary")
    bc.apply(x)
    return firedrake.Mesh(x)

In [ ]:
mesh = create_geometry(num_levels=3, radius=1.0)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots()
axes.set_aspect("equal")
firedrake.triplot(mesh, axes=axes);

In [ ]:
Q = firedrake.FunctionSpace(mesh, "Bernstein", 2)

In [ ]:
from firedrake import max_value, sqrt, inner, as_vector, Constant

def make_obstacle(mesh, obstacle_radius, obstacle_height):
    x = firedrake.SpatialCoordinate(mesh)
    ρ = Constant(obstacle_radius)
    a = Constant(obstacle_height)
    return a * max_value(0, 1 - inner(x, x) / ρ**2)

g_expr = make_obstacle(mesh, obstacle_radius=1/4, obstacle_height=1/4)
g = project(g_expr, Q)

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
fig = plt.figure()
axes = fig.add_subplot(projection='3d')
axes.set_box_aspect((1.0, 1.0, 1/4))
firedrake.trisurf(g, axes=axes);

In [ ]:
import numpy as np
import scipy.optimize

def free_boundary_radius(obstacle_radius, radius):
    def fn(s):
        z = s / obstacle_radius
        return 2 * z**2 * np.log(radius / s) - 1 + z**2

    result = scipy.optimize.root_scalar(fn, x0=1e-3, x1=obstacle_radius)
    return result.root

def make_exact_solution(mesh, obstacle_radius, obstacle_height, radius):
    s = Constant(free_boundary_radius(obstacle_radius, radius))
    R = Constant(radius)
    ρ = Constant(obstacle_radius)
    α = Constant(obstacle_height)
    x = firedrake.SpatialCoordinate(mesh)
    u_expr = α * s**2 / ρ**2 * firedrake.ln(R**2 / inner(x, x))
    return firedrake.conditional(inner(x, x) < s**2, g_expr, u_expr)

u_exact_expr = make_exact_solution(
    mesh, obstacle_radius=1/4, obstacle_height=1/4, radius=1.0
)
u_exact = project(u_exact_expr, Q)

In [ ]:
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
fig = plt.figure(figsize=(16, 12))
axes = fig.add_subplot(projection='3d')
axes.set_box_aspect((1.0, 1.0, 0.25))
firedrake.trisurf(u_exact, axes=axes);

In [ ]:
from firedrake import grad, dx

u = firedrake.Function(Q)
J = 0.5 * inner(grad(u), grad(u)) * dx

In [ ]:
bcs = firedrake.DirichletBC(Q, 0, "on_boundary")
F = firedrake.derivative(J, u)
problem = firedrake.NonlinearVariationalProblem(F, u, bcs)
params = {
    "solver_parameters": {
        "snes_type": "vinewtonrsls",
        "ksp_type": "gmres",
        "pc_type": "lu",
    }
}
solver = firedrake.NonlinearVariationalSolver(problem, **params)

In [ ]:
upper = firedrake.Function(Q)
upper.assign(Constant(10e3));

In [ ]:
solver.solve(bounds=(g, upper))

In [ ]:
fig = plt.figure(figsize=(16, 12))
axes = fig.add_subplot(projection='3d')
axes.set_box_aspect((1.0, 1.0, 0.25))
firedrake.trisurf(u, axes=axes);

In [ ]:
δu = firedrake.project(u - u_exact_expr, Q)

fig, axes = plt.subplots()
axes.set_aspect("equal")
colors = firedrake.tripcolor(δu, axes=axes)
fig.colorbar(colors);

In [ ]:
firedrake.norm(u - u_exact_expr) / firedrake.norm(u_exact_expr)